# Microsoft Foundry Hosted Agents: Build + Deploy from Notebook

This notebook demonstrates the **hosted agents** execution model (containerized, code-first) using the flow from the Microsoft Learn hosted agents page.

It covers:
1) Local test using the hosting adapter (REST on `localhost:8088`)
2) Build + push Docker image to Azure Container Registry (ACR)
3) Capability host setup (preview requirement)
4) Create hosted agent version (container image) via Azure AI Projects SDK
5) Start/stop/update/list deployment via `az cognitiveservices agent ...`
6) Invoke the hosted agent via Responses-compatible API

> Notes
- This is **not** the classic prompt-based agent. Your existing `AgentsClient` code is included later for contrast.
- Hosted agents require **Docker + ACR**, and appropriate **RBAC**.
- Hosted agents are in preview; region/limits apply.


## 0) Prereqs & Configuration

Fill these values before running.


In [ ]:
import os
from pathlib import Path

# ===== Required: Foundry Project Endpoint =====
# Format: https://<AIFoundryResourceName>.services.ai.azure.com/api/projects/<ProjectName>
PROJECT_ENDPOINT = os.getenv("AZURE_AI_PROJECT_ENDPOINT") or os.getenv("PROJECT_ENDPOINT") or ""

# ===== Required: Subscription & Resource Context =====
SUBSCRIPTION_ID = os.getenv("AZ_SUBSCRIPTION_ID") or ""
RESOURCE_GROUP  = os.getenv("AZ_RESOURCE_GROUP") or ""

# Foundry account (Cognitive Services account name) and project name
FOUNDRY_ACCOUNT_NAME = os.getenv("FOUNDRY_ACCOUNT_NAME") or ""
FOUNDRY_PROJECT_NAME = os.getenv("FOUNDRY_PROJECT_NAME") or ""

# ===== ACR =====
ACR_NAME = os.getenv("ACR_NAME") or ""  # e.g. myregistry (no .azurecr.io)
IMAGE_NAME = os.getenv("IMAGE_NAME") or "my-hosted-agent"
IMAGE_TAG  = os.getenv("IMAGE_TAG") or "v1"

# Full image reference like: myregistry.azurecr.io/my-hosted-agent:v1
ACR_LOGIN_SERVER = f"{ACR_NAME}.azurecr.io" if ACR_NAME else ""
IMAGE_REF = f"{ACR_LOGIN_SERVER}/{IMAGE_NAME}:{IMAGE_TAG}" if ACR_LOGIN_SERVER else ""

# ===== Hosted Agent Identity =====
HOSTED_AGENT_NAME = os.getenv("HOSTED_AGENT_NAME") or "ozgur-hosted-agent"

# ===== Compute sizing (examples from docs) =====
HOSTED_CPU = os.getenv("HOSTED_CPU") or "1"
HOSTED_MEMORY = os.getenv("HOSTED_MEMORY") or "2Gi"

# ===== Protocol version =====
# Docs show RESPONSES protocol, version "v1"
CONTAINER_PROTOCOL = "RESPONSES"
CONTAINER_PROTOCOL_VERSION = "v1"

# ===== Model / any app-level env vars your agent container expects =====
# For hosted agents, the model selection often happens in your agent code.
MODEL_DEPLOYMENT_NAME = os.getenv("MODEL_DEPLOYMENT_NAME") or os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME") or ""

# This notebook writes a minimal sample agent app into ./hosted_agent_app
APP_DIR = Path("hosted_agent_app")

def _assert_required():
    missing = []
    for k, v in {
        "PROJECT_ENDPOINT": PROJECT_ENDPOINT,
        "SUBSCRIPTION_ID": SUBSCRIPTION_ID,
        "RESOURCE_GROUP": RESOURCE_GROUP,
        "FOUNDRY_ACCOUNT_NAME": FOUNDRY_ACCOUNT_NAME,
        "FOUNDRY_PROJECT_NAME": FOUNDRY_PROJECT_NAME,
        "ACR_NAME": ACR_NAME,
        "MODEL_DEPLOYMENT_NAME": MODEL_DEPLOYMENT_NAME,
    }.items():
        if not v:
            missing.append(k)
    if missing:
        raise ValueError(f"Missing required config: {', '.join(missing)}")

_assert_required()
print("Config looks set.")
print("PROJECT_ENDPOINT:", PROJECT_ENDPOINT)
print("IMAGE_REF:", IMAGE_REF)


## 1) Install Python dependencies

We install:
- `azure-ai-projects` (for hosted agent version creation & invocation)
- `azure-identity`
- Hosting adapter packages (AgentServer) to run locally on `localhost:8088`


In [ ]:
# If you're on a clean env, uncomment.
# %pip install -U pip

# Azure AI Projects SDK (docs show a preview/beta in some sections)
# If you need the exact beta: %pip install --pre azure-ai-projects==2.0.0b2
%pip install -U azure-ai-projects azure-identity python-dotenv requests

# Hosting adapter packages (Python)
# These wrap your agent code into a Foundry-compatible HTTP service.
%pip install -U azure-ai-agentserver-core azure-ai-agentserver-agentframework


## 2) Azure CLI login + sanity checks

We assume you have:
- `az` installed
- Docker installed

If you want the **azd** path instead of SDK path, you can do it too—but this notebook uses the **SDK + CLI** sequence (it’s the most explicit and debuggable).


In [ ]:
import subprocess

def sh(cmd: str) -> str:
    """Run a shell command and return stdout. Raises on failure."""
    print(f"$ {cmd}")
    out = subprocess.check_output(cmd, shell=True, text=True)
    return out.strip()

print(sh("az account show --query name -o tsv"))
print(sh("az account show --query id -o tsv"))
print(sh("az version --query '"'"'azure-cli'"'"' -o tsv"))
print(sh("docker --version"))


## 3) Write a minimal hosted-agent app (Agent Framework + Hosting Adapter)

This creates a tiny agent service that:
- runs on `localhost:8088`
- exposes `/responses`
- uses your Foundry project endpoint + model deployment

This is the **same mental model** as your existing `AgentsClient` script, but *now the agent is a web service* that can be containerized and hosted.


In [ ]:
APP_DIR.mkdir(exist_ok=True)

# Minimal agent service app using the hosting adapter.
# IMPORTANT: this is a *template* because hosting adapter APIs can evolve.
# If you already have your agent-framework code, drop it into this directory.

(APP_DIR / "agent_app.py").write_text(
    '''\
"""Hosted Agent app (Python) - minimal skeleton.

This file is meant to run locally via hosting adapter on localhost:8088.
Later, it gets containerized for Foundry hosted agents.
"""

import os
from dotenv import load_dotenv

# NOTE: The exact import path can vary by package version.
# The hosting adapter concept is: wrap an agent into a Foundry-compatible HTTP service.
from azure.ai.agentserver.agentframework import from_agentframework

# Your agent logic can be as simple or complex as you want.
# Here we define an ultra-minimal "agent" callable.
def my_agent(user_text: str) -> str:
    return f"You said: {user_text}. (Hosted agent skeleton)"

def main():
    load_dotenv()
    # The adapter will start an HTTP server with Foundry-compatible endpoints.
    # Docs conceptually show one-line hosting like: from_langgraph(my_agent).run()
    # For Agent Framework wrapper we use from_agentframework.
    server = from_agentframework(my_agent)
    server.run(host="0.0.0.0", port=8088)

if __name__ == "__main__":
    main()
    ''',
    encoding="utf-8",
)

# .env for local run
(APP_DIR / ".env").write_text(
    f"""\
AZURE_AI_PROJECT_ENDPOINT={PROJECT_ENDPOINT}
MODEL_DEPLOYMENT_NAME={MODEL_DEPLOYMENT_NAME}
""",
    encoding="utf-8",
)

print("Wrote hosted agent app to:", APP_DIR)
print("Files:", [p.name for p in APP_DIR.iterdir()])


## 4) Run locally (localhost:8088) + smoke test `/responses`

This validates:
- your agent code runs
- the hosting adapter exposes a Foundry-compatible Responses endpoint


In [ ]:
# Start the local server in the background.
# In Jupyter, the simplest approach is to run it in a separate terminal.
# Here we print the command you should run.

print("Run in a terminal:")
print(f"cd {APP_DIR} && python agent_app.py")

print("Then test from another terminal:")
print("curl -s http://localhost:8088/responses -H 'Content-Type: application/json' \\")
print("  -d '{\"input\": {\"messages\": [{\"role\": \"user\", \"content\": \"Where is Seattle?\"}]}}' | jq")


## 5) Create a Dockerfile + build the image

This containerizes the agent web service so Foundry can host it.


In [ ]:
# Minimal Dockerfile. Adjust if you have extra deps.
(APP_DIR / "Dockerfile").write_text(
"""\
FROM python:3.11-slim

WORKDIR /app

# Install runtime deps
COPY . /app

RUN pip install --no-cache-dir -U pip \
    && pip install --no-cache-dir -U python-dotenv azure-identity azure-ai-projects \
    && pip install --no-cache-dir -U azure-ai-agentserver-core azure-ai-agentserver-agentframework

EXPOSE 8088

CMD ["python", "agent_app.py"]
""",
    encoding="utf-8",
)

print("Dockerfile written.")
print("Building docker image...")
sh(f"docker build -t {IMAGE_NAME}:{IMAGE_TAG} {APP_DIR}")
print("Local image built:", f"{IMAGE_NAME}:{IMAGE_TAG}")


## 6) Push image to ACR

Hosted agents require the image to be in Azure Container Registry.


In [ ]:
print("Logging into ACR...")
sh(f"az acr login --name {ACR_NAME}")

print("Tagging + pushing...")
sh(f"docker tag {IMAGE_NAME}:{IMAGE_TAG} {IMAGE_REF}")
sh(f"docker push {IMAGE_REF}")

print("Pushed:", IMAGE_REF)


## 7) Grant ACR pull permissions to the Foundry Project Managed Identity

The docs describe:
- find the project identity principal ID
- assign `Container Registry Repository Reader` on the registry

This step is tenant/RBAC dependent. Below is a **CLI-friendly** skeleton.

> You must fill in or derive the correct principalId for your Foundry project identity.


In [ ]:
# ===== YOU MUST SET THIS =====
# This is the managed identity principal object id for your Foundry project.
# In portal: Foundry project -> Identity -> System assigned -> Object (principal) ID
FOUNDRY_PROJECT_PRINCIPAL_ID = os.getenv("FOUNDRY_PROJECT_PRINCIPAL_ID") or ""
if not FOUNDRY_PROJECT_PRINCIPAL_ID:
    print("Set FOUNDRY_PROJECT_PRINCIPAL_ID env var before running this cell.")
else:
    # Get ACR resource ID
    acr_id = sh(f"az acr show -n {ACR_NAME} --query id -o tsv")
    print("ACR ID:", acr_id)

    # Assign pull permissions
    # Role name may vary; docs reference 'Container Registry Repository Reader'
    # If this role name isn't recognized, list ACR roles and pick the correct one.
    sh(
        f"az role assignment create --assignee-object-id {FOUNDRY_PROJECT_PRINCIPAL_ID} "
        f"--assignee-principal-type ServicePrincipal "
        f"--role 'AcrPull' --scope {acr_id}"
    )
    print("Assigned AcrPull to Foundry project identity.")


## 8) Create Capability Host (preview requirement)

Per docs: you may need an account-level capability host with `enablePublicHostingEnvironment=true`.

This uses a management-plane `PUT` with an access token.


In [ ]:
# Acquire ARM token
token = sh("az account get-access-token --resource https://management.azure.com/ --query accessToken -o tsv")

# Construct URL
api_version = "2025-10-01-preview"  # from docs
caphost_name = "accountcaphost"     # from docs example

url = (
    f"https://management.azure.com/subscriptions/{SUBSCRIPTION_ID}"
    f"/resourceGroups/{RESOURCE_GROUP}"
    f"/providers/Microsoft.CognitiveServices/accounts/{FOUNDRY_ACCOUNT_NAME}"
    f"/capabilityHosts/{caphost_name}"
    f"?api-version={api_version}"
)

payload = {
    "properties": {
        "capabilityHostKind": "Agents",
        "enablePublicHostingEnvironment": True,
    }
}

import requests
resp = requests.put(
    url,
    headers={"content-type": "application/json", "authorization": f"Bearer {token}"},
    json=payload,
)
print("Status:", resp.status_code)
print(resp.text[:2000])


## 9) Create Hosted Agent Version (ImageBasedHostedAgentDefinition)

This registers your container image as a hosted agent version in Foundry.

We use `AIProjectClient` and `create_version()` as shown in the docs.


In [ ]:
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import (
    ImageBasedHostedAgentDefinition,
    ProtocolVersionRecord,
    AgentProtocol,
)

cred = DefaultAzureCredential()
client = AIProjectClient(endpoint=PROJECT_ENDPOINT, credential=cred)

# IMPORTANT:
# Your container might need env vars. Put only what your container expects.
env_vars = {
    "AZURE_AI_PROJECT_ENDPOINT": PROJECT_ENDPOINT,
    "MODEL_DEPLOYMENT_NAME": MODEL_DEPLOYMENT_NAME,
}

agent_version = client.agents.create_version(
    agent_name=HOSTED_AGENT_NAME,
    description="Hosted agent created from notebook",
    definition=ImageBasedHostedAgentDefinition(
        container_protocol_versions=[
            ProtocolVersionRecord(protocol=AgentProtocol.RESPONSES, version=CONTAINER_PROTOCOL_VERSION)
        ],
        cpu=HOSTED_CPU,
        memory=HOSTED_MEMORY,
        image=IMAGE_REF,
        environment_variables=env_vars,
    ),
)

print("Created hosted agent version.")
print("Agent:", agent_version.name if hasattr(agent_version, "name") else HOSTED_AGENT_NAME)
print("Version object:", agent_version)


## 10) Start / Manage the Hosted Agent Deployment (Azure CLI)

Docs show `az cognitiveservices agent start/stop/update/list-versions/show`.

> You need the `agent-version` number. If the SDK object didn’t print it clearly, use `list-versions`.


In [ ]:
# List versions to find the version number
print(sh(
    f"az cognitiveservices agent list-versions "
    f"--account-name {FOUNDRY_ACCOUNT_NAME} "
    f"--project-name {FOUNDRY_PROJECT_NAME} "
    f"--name {HOSTED_AGENT_NAME} -o json"
))


In [ ]:
# ===== SET THE VERSION YOU WANT TO START =====
AGENT_VERSION_TO_START = os.getenv("AGENT_VERSION_TO_START") or "1"

print(sh(
    f"az cognitiveservices agent start "
    f"--account-name {FOUNDRY_ACCOUNT_NAME} "
    f"--project-name {FOUNDRY_PROJECT_NAME} "
    f"--name {HOSTED_AGENT_NAME} "
    f"--agent-version {AGENT_VERSION_TO_START} "
    f"-o json"
))


## 11) Invoke the Hosted Agent (Responses-compatible)

Hosted agents expose an OpenAI Responses-compatible API.

We retrieve the agent, then call `responses.create()` with an `agent` reference.


In [ ]:
from azure.ai.projects.models import AgentReference

agent = client.agents.retrieve(agent_name=HOSTED_AGENT_NAME)
openai_client = client.get_openai_client()

resp = openai_client.responses.create(
    input=[{"role": "user", "content": "Write a haiku about Azure AI Foundry hosted agents."}],
    extra_body={"agent": AgentReference(name=agent.name, version=AGENT_VERSION_TO_START).as_dict()},
)

print(resp.output_text)


## 12) (Optional) Keep your classic prompt-based agent client for comparison

This is your original approach (agent definition executed inside the classic Foundry agent runtime). It does **not** deploy code.

Hosted agents = your code deployed as a container.


In [ ]:
# --- Your original classic script (kept as reference) ---
import os
from dotenv import load_dotenv

from azure.ai.agents import AgentsClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import CodeInterpreterTool

load_dotenv()

def classic_main() -> None:
    project_endpoint = os.getenv("PROJECT_ENDPOINT") or os.getenv("AZURE_AI_PROJECT_ENDPOINT")
    if not project_endpoint:
        raise ValueError("Missing PROJECT_ENDPOINT (or AZURE_AI_PROJECT_ENDPOINT).")

    model_deployment = os.getenv("MODEL_DEPLOYMENT_NAME") or os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
    if not model_deployment:
        raise ValueError("Missing MODEL_DEPLOYMENT_NAME (or AZURE_OPENAI_DEPLOYMENT_NAME).")

    credential = DefaultAzureCredential()
    agents_client = AgentsClient(endpoint=project_endpoint, credential=credential)

    with agents_client:
        code_interpreter = CodeInterpreterTool()

        agent = agents_client.create_agent(
            model=model_deployment,
            name="Quickstart",
            instructions="Be concise.",
            tools=code_interpreter.definitions,
            tool_resources=code_interpreter.resources,
        )
        print(f"Created classic agent: {agent.id}")

        thread = agents_client.threads.create()
        agents_client.messages.create(
            thread_id=thread.id,
            role="user",
            content="Write a haiku about Azure AI Foundry.",
        )

        run = agents_client.runs.create_and_process(
            thread_id=thread.id,
            agent_id=agent.id,
        )
        print(f"Run status: {run.status}")

        messages = agents_client.messages.list(thread_id=thread.id)
        for m in messages:
            print(f"{m.role}: {m.content}")

# classic_main()


## 13) Cleanup

To stop / delete the hosted deployment:

- `az cognitiveservices agent stop ...`
- `az cognitiveservices agent delete-deployment ...`
- `az cognitiveservices agent delete ...`

And optionally delete ACR image/tag.
